# Clustering Crypto

In [109]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [110]:
# Load the crypto_data.csv dataset.
file_load = 'crypto_data.csv'
dataload = pd.read_csv(file_load, index_col=0)
df = pd.DataFrame(dataload)
df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [111]:
# Keep all the cryptocurrencies that are being traded.
df = df.loc[df['IsTrading']==True]
df.head(30)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [112]:
# Remove the "IsTrading" column. 
df = df.drop(columns='IsTrading')
df.shape

(1144, 5)

In [113]:
# Remove rows that have at least 1 null value.
df = df.dropna()
df.shape

(685, 5)

In [114]:
# Keep the rows where coins are mined.
df = df.loc[df['TotalCoinsMined'] > 0]
df.shape

(532, 5)

In [115]:
# Create a new DataFrame that holds only the cryptocurrencies names.
df_crypto = df['CoinName']
df_crypto.shape

(532,)

In [116]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
df = df.drop('CoinName', axis=1)
df.shape

(532, 4)

In [117]:
# Use get_dummies() to create variables for text features.
df.dtypes
df_encoded = pd.get_dummies(df, columns=['Algorithm', 'ProofType'])
df_encoded.shape

(532, 98)

In [118]:
# Standardize the data with StandardScaler().
df_scaled = StandardScaler().fit_transform(df_encoded)
df_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [119]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
df_pca = pca.fit_transform(df_scaled)
df_pca

array([[-0.33157282,  0.99826956, -0.57407482],
       [-0.31496331,  0.9984039 , -0.57442105],
       [ 2.30058236,  1.64899826, -0.66971   ],
       ...,
       [ 0.3264714 , -2.30457445,  0.38808029],
       [-0.16701627, -2.01296347,  0.31615677],
       [-0.29343626,  0.77009983, -0.26995743]])

In [120]:
# Create a DataFrame with the three principal components.
df_PC = pd.DataFrame(data=df_pca, columns=['PC1', 'PC2', 'PC3'],index=df.index)
df_PC.head()

,PC1,PC2,PC3
42,-0.331573,0.998270,-0.574075
404,-0.314963,0.998404,-0.574421
1337,2.300582,1.648998,-0.669710
BTC,-0.137804,-1.311972,0.199714
ETH,-0.157355,-1.960771,0.363505


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [121]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1,10))

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_PC)
    inertia.append(km.inertia_)
    
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

C:\Users\ecreg\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [122]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(df_PC)
# Predict clusters
predictions = model.predict(df_PC)

predictions

array([0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,

In [123]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = df.join(df_PC, how='inner')
clustered_df = clustered_df.join(df_crypto, how='inner')
clustered_df['Class'] = predictions

print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.331573,0.998270,-0.574075,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.314963,0.998404,-0.574421,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.300582,1.648998,-0.669710,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.137804,-1.311972,0.199714,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.157355,-1.960771,0.363505,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.164371,-1.149985,-0.019856,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.401081,1.253369,-0.483995,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.158663,-2.209238,0.410835,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.155801,-1.960873,0.363487,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.167015,-2.012963,0.316157,ZCash,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [124]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df, x='PC1', y='PC2', z='PC3', hover_name='CoinName',hover_data=['Algorithm'], color='Class', symbol='Class', width=900)
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [125]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table()

:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class]

In [126]:
# Print the total number of tradable cryptocurrencies.
trade_count = clustered_df['CoinName'].count()
print (f"There are {trade_count} tradable cryptocurrencies.")

There are 532 tradable cryptocurrencies.


In [127]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
X_scaled = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']]
X_scaled = MinMaxScaler().fit_transform(X_scaled)
X_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [132]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(X_scaled, columns=['TotalCoinSupply', 'TotalCoinsMined'], index=clustered_df.index)
# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName'] = clustered_df['CoinName']
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['Class'] = clustered_df['Class']
plot_df.head(40)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000e+00,42 Coin,0
404,5.320000e-04,1.065855e-03,404Coin,0
1337,3.141593e-01,2.957551e-02,EliteCoin,0
BTC,2.100000e-05,1.810842e-05,Bitcoin,1
ETH,0.000000e+00,1.087731e-04,Ethereum,1
LTC,8.400000e-05,6.367669e-05,Litecoin,1
DASH,2.200000e-05,9.122581e-06,Dash,0
XMR,0.000000e+00,1.737505e-05,Monero,1
ETC,2.100000e-04,1.145060e-04,Ethereum Classic,1
ZEC,2.100000e-05,7.457675e-06,ZCash,1


In [138]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x='TotalCoinsMined', y='TotalCoinSupply', by='Class', hover_cols='CoinName')

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)